In [295]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [296]:
df_input=pd.read_csv('./input_fluxo_caixa.csv',header=None,index_col=0)
df_input=df_input.rename(columns={1:'valor_variavel'})

In [297]:
valor_imovel=df_input.loc['valor_imovel','valor_variavel']
entrada_pp=df_input.loc['entrada_pp','valor_variavel']
entrada_por_valor=df_input.loc['entrada_por_valor','valor_variavel']
financiamento_0_ou_1=df_input.loc['financiamento_0_ou_1','valor_variavel']
area=df_input.loc['area','valor_variavel']

# Custos Iniciais

In [298]:
custo_itbi=0.03*valor_imovel
custo_escritura=df_input.loc['custo_escritura','valor_variavel']
custo_registro=df_input.loc['custo_registro','valor_variavel']


# Cálculo financiamento

In [299]:
# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+encargos_finaciamento


In [300]:
if financiamento_0_ou_1==1:
    n_parcelas=int(df_input.loc['n_parcelas','valor_variavel'])                         # 35 anos
    df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))
    juros_nominal=df_input.loc['juros_nominal','valor_variavel']                        # a.a    
    juros_tr=df_input.loc['tr','valor_variavel']                        # a.a
    juros_nominal=juros_nominal+juros_tr
    custo_avaliacao=df_input.loc['custo_avaliacao','valor_variavel']
    juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

    # he usual way to test for a NaN is to see if it's equal to itself:
    if entrada_por_valor!=entrada_por_valor:
        # Não digitou o valor da entrada
        if entrada_pp!=entrada_pp:
            print('Digite o valor da entrada ou seu percentual.')
        else:
            valor_entrada=valor_imovel*entrada_pp
    else:
        valor_entrada=entrada_por_valor
    
    valor_finaciado=valor_imovel-valor_entrada
    encargos_finaciamento=df_input.loc['encargos_finaciamento','valor_variavel']
    amortizacao=valor_finaciado/n_parcelas
    saldo_devedor=valor_finaciado

else:
    amortizacao=0
    saldo_devedor=0
    n_parcelas=0   
    juros_nominal=0
    custo_avaliacao=0
    juros_efetivo=0  
    valor_finaciado=0
    encargos_finaciamento=0
    valor_entrada=valor_imovel

In [301]:
total_custos_iniciais=valor_entrada+custo_itbi+custo_escritura+custo_registro+custo_avaliacao

In [302]:
for i in range(0,max(24,n_parcelas+1)):             # Se não for financiado seta o df para 24 linhas inicialmente
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)

# Custos fixos

In [303]:
condominio=df_input.loc['condominio','valor_variavel']
iptu_am=df_input.loc['iptu_aa','valor_variavel']/12
luz=df_input.loc['luz','valor_variavel']

total_custos_fixos=condominio+iptu_am+luz

In [304]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0

# Custos com reforma

In [305]:
obra=df_input.loc['obra','valor_variavel']
prazo_obra=df_input.loc['prazo_obra','valor_variavel']                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)

# Cenários de venda

In [306]:
valor_venda=df_input.loc['valor_venda','valor_variavel']
prazo_venda=int(prazo_obra+df_input.loc['prazo_venda_pos_obra','valor_variavel'])                # Prazo da obra em mesês + venda
comissao=df_input.loc['comissao','valor_variavel']

In [307]:
def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir da venda
for i in range(0,len(df)):
    if i>prazo_venda:
        df.loc[i,'total']=0
    elif i==prazo_venda:
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']

Comissão: 7500.0


In [308]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

df=df[df['total']!=0]
df.to_csv('output.csv', header=True, float_format='%.2f')
df

,parcela,saldo_devedor,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0,"100,000.00",771.67,"575,600.00",0.00,0.00,"576,371.67",2022-08-23
1,"1,270.60","99,761.90",771.67,0.00,"7,500.00",0.00,"9,542.26",2022-09-23
2,"1,268.49","99,523.81",771.67,0.00,"7,500.00",0.00,"9,540.16",2022-10-23
3,"1,266.39","99,285.71",771.67,0.00,0.00,0.00,"2,038.06",2022-11-23
4,"1,264.29","99,047.62",771.67,0.00,0.00,0.00,"2,035.96",2022-12-23
5,"1,262.19","98,809.52",771.67,0.00,0.00,0.00,"2,033.86",2023-01-23
6,"1,260.09","98,571.43",771.67,0.00,0.00,0.00,"2,031.76",2023-02-23
7,"1,257.99","98,333.33",771.67,0.00,0.00,0.00,"2,029.65",2023-03-23
8,"1,255.89","98,095.24",771.67,0.00,0.00,0.00,"2,027.55",2023-04-23
9,"1,253.79","97,857.14",771.67,0.00,0.00,0.00,"2,025.45",2023-05-23


In [309]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Lucro: ',-x['total'].sum())
    # return (t_aa*100,t_am*100,-x['total'].sum())

In [310]:
cal_irr(df)

TII a.a.(%):  3.6927795477003054
TII a.m.(%):  0.30264286282903896
Lucro:  26061.208333333256


In [311]:
valor_m2_aquisicao=valor_imovel/area
print(f'm2 aquisição: {valor_m2_aquisicao}')
valor_m2_venda=valor_venda/area
print(f'm2 reforma: {obra/area}')
print(f'm2 venda: {valor_m2_venda}')


m2 aquisição: 10156.25
m2 reforma: 234.375
m2 venda: 11718.75


In [312]:
11*66+70

796

In [313]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda